In [4]:
from __future__ import division

import import_ipynb
import time
import torch 
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import cv2 
from util import *
import argparse
import os 
import os.path as osp
from model import Network
from img_process import preprocess_img, inp_to_image
import pandas as pd
import random 
import pickle as pkl

In [3]:
def arg_parse():

    parser = argparse.ArgumentParser(description='YOLOv4 ')
   
    parser.add_argument("--images", dest = 'images', help = 
                        "Image / Directory containing input images",
                        default = "images", type = str)
    parser.add_argument("--result", dest = 'result', help = 
                        " Directory to store results ",
                        default = "result", type = str)
    parser.add_argument("--nms_thresh", dest = "nms_thresh", help = "NMS Threshhold", default = 0.4)

    parser.add_argument("--bs", dest = "bs", help = "Batch size", default = 1)
    parser.add_argument("--confidence", dest = "confidence", help = "Detection Confidence ", default = 0.5)
    parser.add_argument("--cfg", dest = 'configfile', help = 
                        "Config file",
                        default = "cfg/yolov4.cfg", type = str)
    parser.add_argument("--weights", dest = 'weightsfile', help = 
                        "weightsfile",
                        default = "C:/Users/User/weights/yolov4.weights", type = str)
    parser.add_argument("--reso", dest = 'resolution', help = 
                        "Input resolution of the network",
                        default = "608", type = str)
    parser.add_argument("--scales", dest = "scales", help = "Scales to use for detection",
                        default = "1,2,3", type = str)   
    parser.add_argument("-f") # Quick fix for Jupyter Notebook
    return parser.parse_args()
    
#     args = ("--images", dest = 'images', help = "Image / Directory containing input images",default = "images", type = str,
#             "--result", dest = 'result', help = " Directory to store results ", default = "result", type = str,
#             "--nms_thresh", dest = "nms_thresh", help = "NMS Threshhold", default = 0.4, 
#             "--bs", dest = "bs", help = "Batch size", default = 1, "--confidence", dest = "confidence", help = "Detection Confidence ", default = 0.5,
#             "--cfg", dest = 'configfile', help = "Config file", default = "cfg/yolov3.cfg", type = str,
#             "--weights", dest = 'weightsfile', help = "weightsfile", default = "C:/Users/User/weights/yolov4.weights", type = str,
#             "--reso", dest = 'resolution', help = "Input resolution of the network", default = "256", type = str,
#             "--scales", dest = "scales", help = "Scales to use for detection", default = "1,2,3", type = str)
    
#     return arg_parse(*args)
    

if __name__ ==  '__main__':
    args = arg_parse()
    
    scales = args.scales
    print(scales)
 
    images = args.images
    batch_size = int(args.bs)
    confidence = float(args.confidence)
    nms_thesh = float(args.nms_thresh)
    start = 0

    CUDA = torch.cuda.is_available()

    num_classes = 70
    classes = load_classes('data/coco.names') 

    model = Network(args.configfile)
    model.load_weights(args.weightsfile)
    print("Network loaded")
    
    model.dn_info["height"] = args.resolution
    in_dim = int(model.dn_info["height"])

    if CUDA:
        model.cuda()
    model.eval()
    
    read_dir = time.time()
    try:
        imlist = [osp.join(osp.realpath('.'), images, img) for img in os.listdir(images) if os.path.splitext(img)[1] == '.png' or os.path.splitext(img)[1] =='.jpeg' or os.path.splitext(img)[1] =='.jpg']
    except NotADirectoryError:
        imlist = []
        imlist.append(osp.join(osp.realpath('.'), images))
    except FileNotFoundError:
        print ("No with the name {}".format(images))
        exit()
        
    if not os.path.exists(args.result):
        os.makedirs(args.result)
            
    batches = list(map(preprocess_img, imlist, [in_dim for x in range(len(imlist))]))
    im_batches = [x[0] for x in batches]
    orig_ims = [x[1] for x in batches]
    im_dim_list = [x[2] for x in batches]
    #Explain
    im_dim_list = torch.FloatTensor(im_dim_list).repeat(1,2)
    
    
    
    if CUDA:
        im_dim_list = im_dim_list.cuda()
    
    leftover = 0
    
    if (len(im_dim_list) % batch_size):
        leftover = 1
        

    i = 0
    

    write = False
    
    
    objs = {}
    
    
    
    for batch in im_batches:
        if CUDA:
            batch = batch.cuda()
        #print('batch size => ', batch.size())
        with torch.no_grad():
            prediction = model(batch, CUDA)
        
        print('prediction: ',prediction.size())
        
        
        prediction = write_results(prediction, confidence, num_classes, nms = True, nms_conf = nms_thesh)

        
        if type(prediction) == int:
            i += 1
            continue

            
        #Add the current batch number
        prediction[:,0] += i*batch_size
        
    
            
          
        if not write:
            output = prediction
            write = 1
        else:
            output = torch.cat((output,prediction))



        for im_num, image in enumerate(imlist[i*batch_size: min((i +  1)*batch_size, len(imlist))]):
            print('im_id: ', im_num)
            print('i: ', i)
            print('batch_size: ', batch_size)
            im_id = i*batch_size + im_num
            objs = [classes[int(x[-1])] for x in output if int(x[0]) == im_id]
            print("{0:20s} {1:s}".format("Objects Detected:", " ".join(objs)))
            print("----------------------------------------------------------")
#             confidences = [classes[int(x[-1])] for x in output if int(x[0]) == im_id]
        i += 1

        
        if CUDA:
            torch.cuda.synchronize()
    
    try:
        output
    except NameError:
        print("No detections were made")
        exit()
        
    im_dim_list = torch.index_select(im_dim_list, 0, output[:,0].long())
    
    scaling_factor = torch.min(in_dim/im_dim_list,1)[0].view(-1,1)
    
    
    output[:,[1,3]] -= (in_dim - scaling_factor*im_dim_list[:,0].view(-1,1))/2
    output[:,[2,4]] -= (in_dim - scaling_factor*im_dim_list[:,1].view(-1,1))/2
    
    
    
    output[:,1:5] /= scaling_factor
    
    for i in range(output.shape[0]):
        output[i, [1,3]] = torch.clamp(output[i, [1,3]], 0.0, im_dim_list[i,0])
        output[i, [2,4]] = torch.clamp(output[i, [2,4]], 0.0, im_dim_list[i,1])
        
#     colors = pkl.load(open("pallete", "rb"))
    with open("pallete", "rb") as pkl_file:
        colors = pkl.load(pkl_file)
    
    def write(x, batches, results):
        c1 = tuple(x[1:3].int())
        c2 = tuple(x[3:5].int())
        img = results[int(x[0])]
        cls = int(x[-1])
        label = "{0}".format(classes[cls])
        color = random.choice(colors)
        cv2.rectangle(img, c1, c2,color, 1)
        t_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_PLAIN, 1 , 1)[0]
        c2 = c1[0] + t_size[0] + 3, c1[1] + t_size[1] + 4 
        cv2.rectangle(img, c1, c2,color, -1)
        cv2.putText(img, label, (c1[0], c1[1] + t_size[1] + 4), cv2.FONT_HERSHEY_PLAIN, 1, [225,255,255], 1)
        return img
    
            
    list(map(lambda x: write(x, im_batches, orig_ims), output))
      
    det_names = pd.Series(imlist).apply(lambda x: "{}/det_{}".format(args.result,x.split("\\")[-1]))
    
    list(map(cv2.imwrite, det_names, orig_ims))

    torch.cuda.empty_cache()
    

1,2,3
['[net]', 'batch=64', 'subdivisions=8', 'width=608', 'height=608', 'channels=3', 'momentum=0.949', 'decay=0.0005', 'angle=0', 'saturation = 1.5', 'exposure = 1.5', 'hue=.1', 'learning_rate=0.0013', 'burn_in=1000', 'max_batches = 500500', 'policy=steps', 'steps=400000,450000', 'scales=.1,.1', 'mosaic=1', '[convolutional]', 'batch_normalize=1', 'filters=32', 'size=3', 'stride=1', 'pad=1', 'activation=mish', '[convolutional]', 'batch_normalize=1', 'filters=64', 'size=3', 'stride=2', 'pad=1', 'activation=mish', '[convolutional]', 'batch_normalize=1', 'filters=64', 'size=1', 'stride=1', 'pad=1', 'activation=mish', '[route]', 'layers = -2', '[convolutional]', 'batch_normalize=1', 'filters=64', 'size=1', 'stride=1', 'pad=1', 'activation=mish', '[convolutional]', 'batch_normalize=1', 'filters=32', 'size=1', 'stride=1', 'pad=1', 'activation=mish', '[convolutional]', 'batch_normalize=1', 'filters=64', 'size=3', 'stride=1', 'pad=1', 'activation=mish', '[shortcut]', 'from=-3', 'activation=li

Network loaded


C:\Users\User\Anaconda3\lib\site-packages\torch\nn\functional.py:3121: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


Size before transform =>  torch.Size([1, 255, 76, 76])
Size after transform =>  torch.Size([1, 17328, 85])
Size before transform =>  torch.Size([1, 255, 38, 38])
Size after transform =>  torch.Size([1, 4332, 85])
Size before transform =>  torch.Size([1, 255, 19, 19])
Size after transform =>  torch.Size([1, 1083, 85])
prediction:  torch.Size([1, 22743, 85])
torch.Size([22743, 1])
image_pred:  torch.Size([22743, 85])
max_conf:  torch.Size([22743, 1])
max_conf_score:  torch.Size([22743, 1])
image_pred:  torch.Size([22743, 7])
im_id:  0
i:  0
batch_size:  1
Objects Detected:    person bed
----------------------------------------------------------
Size before transform =>  torch.Size([1, 255, 76, 76])
Size after transform =>  torch.Size([1, 17328, 85])
Size before transform =>  torch.Size([1, 255, 38, 38])
Size after transform =>  torch.Size([1, 4332, 85])
Size before transform =>  torch.Size([1, 255, 19, 19])
Size after transform =>  torch.Size([1, 1083, 85])
prediction:  torch.Size([1, 22

IndexError: list index out of range

In [4]:
print(prediction, confidence, num_classes)

tensor([[  0., nan., nan.,  ..., nan., nan.,  79.],
        [  0., nan., nan.,  ..., nan., nan.,   0.],
        [  0., nan., nan.,  ..., nan., nan.,   0.],
        ...,
        [  0., nan., nan.,  ..., nan., nan.,   0.],
        [  0., nan., nan.,  ..., nan., nan.,   0.],
        [  0., nan., nan.,  ..., nan., nan.,   0.]], device='cuda:0') 0.5 80


In [3]:
objs = [classes[int(x[-1])] for x in output if int(x[0]) == im_id]

IndexError: list index out of range

In [4]:
print('output: ',output.size())
print('prediction: ',prediction.size())

print(classes)
print(len(classes))
print(prediction)
print(confidence)


output:  torch.Size([384, 8])
prediction:  torch.Size([192, 8])
['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
80
tensor([[  1., nan., nan.,  ..., nan., nan.,  79.],
        [  1., 